In [1]:
import utils
import threading
from datetime import datetime as dt
from playsound import playsound
from configs import API_KEY,BASE_URL,MIN_PERCENT_REQUIRED,BRL_AMOUNT_TRADE
from biscoint import get_rate_limit, get_balance, get_offer, confirm_offer, get_ticker
from coinsamba import get_offers_coinsamba
import time

In [4]:
#Inicial configs
rate_limit = get_rate_limit()
sleep_time_offers = ((rate_limit["windowMs"] / rate_limit["maxRequests"]) / 1000) * 1.5
no_rush_percent = -0.2 # If the spread is lower than this, wait normal time
initial_balance = get_balance()
last_balance = initial_balance
ticker = get_ticker()
amount_btc_to_trade = utils.btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])

In [5]:
def async_offer(op:str,amount:str,isQuote:bool):
    try:
        response = get_offer(op,amount,isQuote)
        request_orders[op] = response
    except Exception as e:
        print("Erro ao rodar thread",e)
# thread_buy = threading.Thread(target=async_offer,args=('buy',AMOUNT,False))
# thread_sell = threading.Thread(target=async_offer,args=('sell',AMOUNT,False))
# thread_buy.start()
# time.sleep(0.1)
# thread_sell.start()
# thread_buy.join()
# thread_sell.join()

In [6]:
while True:
    start_time = dt.now()
    try:
        request_orders = {'buy':None,'sell':None}
        request_orders['buy'] = get_offer('buy',str(amount_btc_to_trade),False)
        request_orders['sell'] = get_offer('sell',str(amount_btc_to_trade),False)

        buy_price = request_orders['buy']['efPrice']
        sell_price = request_orders['sell']['efPrice']
        percent = utils.percent(buy_price, sell_price)

        if percent >= MIN_PERCENT_REQUIRED:
            print(f"Arbitrage oportunity: buy:{buy_price}   sell:{sell_price} ->  {percent}%")
            playsound('beep.wav')
            executed_buy = confirm_offer(buy['offerId'])
            executed_sell = confirm_offer(sell['offerId'])

        print(percent)

        end_time = dt.now()
        seconds_elapsed = (end_time - start_time).total_seconds()
        print(f"Took {seconds_elapsed} seconds")
        print("Wait",sleep_time_offers-percent)
        if percent <= no_rush_percent:
            time.sleep(sleep_time_offers)
        else:
            time.sleep(0.5)
        
    except Exception as e:
        print(e)



-0.722
Took 2.121954 seconds
Wait 3.722
-0.722
Took 1.687451 seconds
Wait 3.722
-0.758
Took 0.933018 seconds
Wait 3.758
-0.972
Took 0.746075 seconds
Wait 3.972


KeyboardInterrupt: 

In [24]:
sell = get_offers_coinsamba('sell',amount_btc_to_trade)
buy = get_offers_coinsamba('buy',amount_btc_to_trade)

calc_percent = utils.percent(sell[0]['averagePrice'],buy[0]['averagePrice'])
print(calc_percent)

-7.499


In [54]:
import requests
COINSAMBA_BASE_URL="https://api.coinsamba.com/v0"
def get_offers_coinsamba(op:str,amount:str):
    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'
    }
    response = requests.get(f'{COINSAMBA_BASE_URL}/bestPrice?refference=quote&side={op}&amount={amount}&base=BTC&quote=BRL',headers=headers)
    return response.json()

get_offers_coinsamba('buy',amount_btc_to_trade)

[{'fullName': 'bitPreço',
  'isSupporter': True,
  'exchangeId': 'bitpreco',
  'totalQuote': 0.0001810908,
  'totalBase': 5.911354555662915e-10,
  'averagePrice': 306344,
  'totalFee': 0,
  'totalFeeBase': 0,
  'liquidity': 564110.54031905,
  'depth': 1,
  'rating': 5},
 {'fullName': 'PagCripto OTC',
  'isSupporter': True,
  'exchangeId': 'pagcripto_otc',
  'totalQuote': 0.0001810908,
  'totalBase': 5.905593236379285e-10,
  'averagePrice': 306642.86,
  'totalFee': 0,
  'totalFeeBase': 0,
  'liquidity': 306949502.86,
  'depth': 1,
  'rating': 0},
 {'fullName': 'Bitrecife',
  'isSupporter': True,
  'exchangeId': 'bitrecife',
  'totalQuote': 0.0001810908,
  'totalBase': 5.904087108691425e-10,
  'averagePrice': 306721.0843373494,
  'totalFee': 0,
  'totalFeeBase': 2.36163484347657e-12,
  'liquidity': 24840.814669896,
  'depth': 1,
  'rating': 5},
 {'fullName': 'Coinext',
  'isSupporter': False,
  'exchangeId': 'coinext',
  'totalQuote': 0.0001810908,
  'totalBase': 5.892732878027645e-10,
 